In [1]:
import pandas as pd
import numpy as np
import spacy
from string import punctuation
import string
from collections import Counter
from numpy import savetxt

In [2]:
data=pd.read_csv("Tweets.csv")
labels=data['airline_sentiment'].to_numpy()
rev=data['text'].to_numpy()

In [3]:
d = np.array([x.lower() if isinstance(x, str) else x for x in rev])
i=0
for txt in d:
    d[i]=' '.join(word for word in txt.split(' ') if not (word.startswith('@') or  word.startswith('http') ))
    d[i] = ''.join([c for c in d[i] if c not in punctuation])
    i=i+1   
d[:20]

array(['what said',
       'plus youve added commercials to the experience tacky',
       'i didnt today must mean i need to take another trip',
       'its really aggressive to blast obnoxious entertainment in your guests faces amp they have little recourse',
       'and its a really big bad thing about it',
       'seriously would pay 30 a flight for seats that didnt have this playing\nits really the only bad thing about flying va',
       'yes nearly every time i fly vx this “ear worm” won’t go away ',
       'really missed a prime opportunity for men without hats parody there',
       'well i didnt…but now i do d',
       'it was amazing and arrived an hour early youre too good to me',
       'did you know that suicide is the second leading cause of death among teens 1024',
       'i lt3 pretty graphics so much better than minimal iconography d',
       'this is such a great deal already thinking about my 2nd trip to amp i havent even gone on my 1st trip yet p',
       'im flying y

In [5]:
nlp = spacy.load('en_core_web_sm')
t_rev=[]
for s in d:
    tokens=nlp(str(s))
    lem=" ".join([token.lemma_ for token in tokens if token.is_stop==False and token.is_alpha==True and token.like_num==False and token.like_url==False and token.like_email==False and token.is_punct==False])
    t_rev.append(lem)
t_rev

['say',
 'plus have add commercial experience tacky',
 'not today mean need trip',
 'aggressive blast obnoxious entertainment guest face amp little recourse',
 'big bad thing',
 'seriously pay flight seat not playing bad thing fly va',
 'yes nearly time fly vx ear worm will away',
 'miss prime opportunity man hat parody',
 'didnt d',
 'amazing arrive hour early good',
 'know suicide second leading cause death teen',
 'pretty graphic well minimal iconography d',
 'great deal think trip amp not go trip p',
 'be fly fabulous seductive sky u stress away travel',
 'thanks',
 'sfopdx schedule mia',
 'excited cross country flight lax mco have hear great thing virgin america',
 'fly nyc sfo week not fully sit seat large gentleman help',
 'fly',
 'know amazingly awesome bosfll want fly',
 'fare time carrier seat available select',
 'love graphic',
 'love hipster innovation feel good brand',
 'make bosgtla non stop permanently anytime soon',
 'guy mess seating reserve seat friend guy give seat a

In [6]:
all_words=[]
reviews=[]
for ls in t_rev:
    tp=ls.split()
    reviews.append(tp)
    for w in tp:
        all_words.append(w)
reviews[:20]

[['say'],
 ['plus', 'have', 'add', 'commercial', 'experience', 'tacky'],
 ['not', 'today', 'mean', 'need', 'trip'],
 ['aggressive',
  'blast',
  'obnoxious',
  'entertainment',
  'guest',
  'face',
  'amp',
  'little',
  'recourse'],
 ['big', 'bad', 'thing'],
 ['seriously',
  'pay',
  'flight',
  'seat',
  'not',
  'playing',
  'bad',
  'thing',
  'fly',
  'va'],
 ['yes', 'nearly', 'time', 'fly', 'vx', 'ear', 'worm', 'will', 'away'],
 ['miss', 'prime', 'opportunity', 'man', 'hat', 'parody'],
 ['didnt', 'd'],
 ['amazing', 'arrive', 'hour', 'early', 'good'],
 ['know', 'suicide', 'second', 'leading', 'cause', 'death', 'teen'],
 ['pretty', 'graphic', 'well', 'minimal', 'iconography', 'd'],
 ['great', 'deal', 'think', 'trip', 'amp', 'not', 'go', 'trip', 'p'],
 ['be',
  'fly',
  'fabulous',
  'seductive',
  'sky',
  'u',
  'stress',
  'away',
  'travel'],
 ['thanks'],
 ['sfopdx', 'schedule', 'mia'],
 ['excited',
  'cross',
  'country',
  'flight',
  'lax',
  'mco',
  'have',
  'hear',
  'gre

In [7]:
counts=Counter(all_words)
lst= sorted(counts,key=counts.get,reverse=True)
mp = {word: cnt for cnt, word in enumerate(lst, 1)}
num_rev=[]
for rv in reviews:
    num_rev.append([mp[wrd] for wrd in rv])
len(lst)

10261

In [8]:
num_lab=[]
for l in labels:
    if l == "negative":
        num_lab.append(0)
    elif l == "neutral":
        num_lab.append(1)
    else:
        num_lab.append(1)
num_lab[:20]

[1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1]

In [9]:
review_lens = Counter([len(x) for x in num_rev])
print("Zero-length reviews: {}".format(review_lens[0]))
print("Maximum review length: {}".format(max(review_lens)))

Zero-length reviews: 68
Maximum review length: 20


In [10]:
print('Number of reviews before removing outliers: ', len(num_rev))
non_zero_idx = [x for x, review in enumerate(num_rev) if len(review) != 0]
num_rev = [num_rev[x] for x in non_zero_idx]
num_lab = np.array([num_lab[x] for x in non_zero_idx])
print('Number of reviews after removing outliers: ', len(num_rev))

Number of reviews before removing outliers:  14640
Number of reviews after removing outliers:  14572


In [11]:
def pad_features(reviews_ints, seq_length):
    features = np.zeros((len(reviews_ints), seq_length), dtype=int)
    for i, row in enumerate(reviews_ints):
        features[i, -len(row):] = np.array(row)[:seq_length]
    return features

In [12]:
num_rev=pad_features(num_rev,max(review_lens))
num_rev

array([[   0,    0,    0, ...,    0,    0,   42],
       [   0,    0,    0, ...,  884,   80, 4441],
       [   0,    0,    0, ...,  193,   18,   84],
       ...,
       [   0,    0,    0, ...,  293,  278,   25],
       [   0,    0,    0, ...,   32,  752, 1667],
       [   0,    0,    0, ...,  389,   64,    1]])

In [13]:
savetxt('review.csv',num_rev,delimiter=',')
savetxt('label.csv',num_lab,delimiter=',')